In [14]:
import wikipediaapi
import pandas as pd




In [29]:
wiki_wiki = wikipediaapi.Wikipedia('hi')
page_py = wiki_wiki.page('ब्रिटेन')

body = page_py.text.replace('\n', '').replace('==', '').replace('\u200d', '').replace('।', '')


In [30]:
def print_langlinks(page):
        langlinks = page.langlinks
        
        for k in sorted(langlinks.keys()):
            v = langlinks[k]
            if(v.language=='en'):
                return v.title
            
        return "Page does not exist"



In [31]:
def print_links(page,body):
    links=[]
    for link in page.links:
        if body.find(link) != -1:
            links.append(link)

    return links
link_list=print_links(page_py,body)

In [32]:
def word_list(page):
    x=[]
    for page in link_list:
        page_1 = wiki_wiki.page(page)
        y=print_langlinks(page_1)
        
        x.append([page,y])
    return x



In [33]:
list=word_list(link_list)
df=pd.DataFrame(list)

In [34]:
df

,0,1
0,इंग्लैंड,England
1,ग्रेट ब्रिटेन,Great Britain
2,ग्रेट ब्रिटेन और आयरलैंड का यूनाइटेड किंगडम,United Kingdom of Great Britain and Ireland
3,ग्रेट ब्रिटेन राजशाही,Kingdom of Great Britain
4,ब्रिटेन (शब्द),Page does not exist
5,यूनाइटेड किंगडम,European Charter for Regional or Minority Lang...
6,वेल्स,Wales
7,स्कॉटलैंड,Scotland
